### Dependencies

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from align_image_code import align_images
from skimage.metrics import mean_squared_error, structural_similarity as ssim

# Project 2 - Fun with Filters and Frequencies

## Part 1 - Fun with Filters

### Part 1.1 - Finite Difference Operators

In [ ]:
# Load the image
cameraman = plt.imread('../data/cameraman.png')
cameraman = np.mean(cameraman, axis = 2)

# Define finite difference operators
D_x = np.array([[1, -1]])
D_y = np.array([[1], [-1]])

# Compute partial derivatives
partial_derivative_x = convolve2d(cameraman, D_x, mode = 'same')
partial_derivative_y = convolve2d(cameraman, D_y, mode = 'same')

# Plot the original image and its partial derivatives
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.imshow(cameraman, cmap='gray')
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(partial_derivative_x, cmap='gray')
plt.title('Partial Derivative X')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(partial_derivative_y, cmap='gray')
plt.title('Partial Derivative Y')
plt.axis('off')

plt.show()

In [ ]:
# Gradient magnitude
gradient_magnitude = np.sqrt(partial_derivative_x**2 + partial_derivative_y**2)

plt.imshow(gradient_magnitude, cmap = 'gray')
plt.title('Gradient Magnitude')
plt.axis('off')
plt.show()

In [ ]:
# Binarize the gradient magnitude image
threshold = 0.2
binary_cameraman_image = gradient_magnitude > threshold

# Plot the original image, gradient magnitude, and binary image
plt.figure(figsize = (12, 4))

plt.subplot(1, 3, 1)
plt.imshow(cameraman, cmap = 'gray')
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(gradient_magnitude, cmap = 'gray')
plt.title('Gradient Magnitude')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(binary_cameraman_image, cmap = 'gray')
plt.title('Binarized Gradient Magnitude')
plt.axis('off')

plt.show()

### Part 1.2 - Derivative of Gaussian (DoG) Filter

In [5]:
def create_gaussian_filter(sigma = 1.0, kernel_size = 6):
    gaussian_1d = cv2.getGaussianKernel(kernel_size, sigma)
    gaussian_2d = np.outer(gaussian_1d, gaussian_1d)
    return gaussian_2d

gaussian_2d = create_gaussian_filter(sigma=1.0, kernel_size=6)

In [ ]:
# Convolve the original image with the Gaussian filter to get a blurred image
blurred_image = convolve2d(cameraman, gaussian_2d, mode = 'same')

partial_derivative_x_blurred = convolve2d(blurred_image, D_x, mode = 'same')
partial_derivative_y_blurred = convolve2d(blurred_image, D_y, mode = 'same')

gradient_magnitude_blurred = np.sqrt(partial_derivative_x_blurred**2 + partial_derivative_y_blurred**2)

threshold = 0.05
binary_edge_cameraman_blurred = gradient_magnitude_blurred > threshold

fig1, axes1 = plt.subplots(1, 2, figsize=(12, 6))

# Plot the partial derivative x blurred image
axes1[0].imshow(partial_derivative_x_blurred, cmap='gray')
axes1[0].set_title('Partial Derivative X of Blurred Image')
axes1[0].axis('off')

# Plot the partial derivative y blurred image
axes1[1].imshow(partial_derivative_y_blurred, cmap='gray')
axes1[1].set_title('Partial Derivative Y of Blurred Image')
axes1[1].axis('off')

plt.tight_layout()
plt.show()

fig2, axes2 = plt.subplots(1, 2, figsize=(12, 6))

# Plot the gradient magnitude blurred image
axes2[0].imshow(gradient_magnitude_blurred, cmap='gray')
axes2[0].set_title('Gradient Magnitude')
axes2[0].axis('off')

# Plot the binarized gradient magnitude image
axes2[1].imshow(binary_edge_cameraman_blurred, cmap='gray')
axes2[1].set_title('Binarized Gradient Magnitude')
axes2[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Create DoG filters by convolving the Gaussian filter with D_x and D_y
DoG_x = convolve2d(gaussian_2d, D_x, mode = 'same')
DoG_y = convolve2d(gaussian_2d, D_y, mode = 'same')

# Display the resulting DoG filters
plt.figure(figsize=(8, 4))

plt.subplot(1, 2, 1)
plt.imshow(DoG_x, cmap='gray')
plt.title('DoG Filter X')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(DoG_y, cmap='gray')
plt.title('DoG Filter Y')
plt.axis('off')

plt.show()

In [ ]:
# Convolve the original image with DoG_x and DoG_y
grad_x_DoG = convolve2d(cameraman, DoG_x, mode = 'same')
grad_y_DoG = convolve2d(cameraman, DoG_y, mode = 'same')

# Compute the gradient magnitude
gradient_magnitude_DoG = np.sqrt(grad_x_DoG**2 + grad_y_DoG**2)

# Display the gradient magnitude image
plt.imshow(gradient_magnitude_DoG, cmap = 'gray')
plt.title('Gradient Magnitude using DoG Filters')
plt.axis('off')
plt.show()

In [ ]:
# Set up the figure to display images side by side
plt.figure(figsize=(12, 6))

# First subplot: Gradient magnitude with Gaussian smoothing + finite difference
plt.subplot(1, 2, 1)
plt.imshow(gradient_magnitude_blurred, cmap = 'gray')
plt.title('Gradient Magnitude (Smoothing + Finite Difference)')
plt.axis('off')

# Second subplot: Gradient magnitude with DoG filters
plt.subplot(1, 2, 2)
plt.imshow(gradient_magnitude_DoG, cmap = 'gray')
plt.title('Gradient Magnitude (DoG Filter)')
plt.axis('off')

# Display the comparison
plt.show()

## Part 2 - Fun with Frequencies

### Part 2.1 - Image "Sharpening"

In [ ]:
taj = plt.imread('../data/taj.jpg')

# Function to apply unsharp mask on each channel
def sharpen_image(image: np.ndarray, sigma: float, alpha: float) -> np.ndarray:
    # Normalize the image to be in the range [0, 1]
    image = image / 255.0
    
    sharpened_image = np.zeros_like(image)
    
    # Create the 1D Gaussian kernel
    kernel_size = int(6 * sigma) + 1  # Kernel size depends on sigma
    gaussian_1d = cv2.getGaussianKernel(kernel_size, sigma)
    
    # Create the 2D Gaussian kernel by taking the outer product
    gaussian_2d = np.outer(gaussian_1d, gaussian_1d)
    
    # Apply unsharp mask to each channel separately
    for i in range(3):  # For R, G, B channels
        blurred_channel = convolve2d(image[:, :, i], gaussian_2d, mode = 'same')
        details = image[:, :, i] - blurred_channel
        sharpened_image[:, :, i] = image[:, :, i] + alpha * details
    
    sharpened_image = np.clip(sharpened_image, 0, 1)
    
    return sharpened_image

sharpened_taj = sharpen_image(taj, sigma = 2.0, alpha = 0.75)

# Display the original and sharpened images side by side for comparison
plt.figure(figsize=(12, 6))

# Original image
plt.subplot(1, 2, 1)
plt.imshow(taj)
plt.title('Original Taj Mahal Image')
plt.axis('off')

# Sharpened image
plt.subplot(1, 2, 2)
plt.imshow(sharpened_taj)
plt.title('Sharpened Taj Mahal Image')
plt.axis('off')

plt.show()


In [ ]:
camera_obscura = plt.imread('../data/camera_obscura.jpg')

sharpened_camera_obscura = sharpen_image(camera_obscura, sigma = 3.0, alpha = 6.0)

# Display the original and sharpened images side by side for comparison
plt.figure(figsize=(12, 6))

# Original image
plt.subplot(1, 2, 1)
plt.imshow(camera_obscura)
plt.title('Original Camera Obscura Image')
plt.axis('off')

# Sharpened image
plt.subplot(1, 2, 2)
plt.imshow(sharpened_camera_obscura)
plt.title('Sharpened Camera Obscura Image')
plt.axis('off')

plt.show()

In [ ]:
midsummer = plt.imread('../data/midsummer.jpg')

sharpened_midsummer = sharpen_image(midsummer, sigma = 2.0, alpha = 2.0)

# Display the original and sharpened images side by side for comparison
plt.figure(figsize=(12, 6))

# Original image
plt.subplot(1, 2, 1)
plt.imshow(midsummer)
plt.title('Original Midsummer Image')
plt.axis('off')

# Sharpened image
plt.subplot(1, 2, 2)
plt.imshow(sharpened_midsummer)
plt.title('Sharpened Midsummer Image')
plt.axis('off')

plt.show()

In [ ]:
# Load the image
lena = plt.imread('../data/lena.tiff')

def create_gaussian_kernel(kernel_size: int, sigma: float) -> np.ndarray:
    if kernel_size % 2 == 0:
        raise ValueError('Kernel size must be an odd number')
                         
    gaussian_1d = cv2.getGaussianKernel(kernel_size, sigma)
    gaussian_2d = np.outer(gaussian_1d, gaussian_1d)
    
    return gaussian_2d

gaussian_2d = create_gaussian_kernel(kernel_size = 15, sigma = 2.0)

# Blur each channel of the image
blurred_lena = np.zeros_like(lena)
for i in range(3):
    blurred_lena[:, :, i] = convolve2d(lena[:, :, i], gaussian_2d, mode='same')

# Sharpen the image again
sharpened_lena = sharpen_image(blurred_lena, sigma = 1.2, alpha = 4.0)

# Compute MSE and SSIM
mse_value = mean_squared_error(lena, sharpened_lena)
ssim_value = ssim(lena, sharpened_lena, multichannel=True, win_size=7, channel_axis=2, data_range=lena.max() - lena.min())

# Display the original, blurred images and sharpened side by side for comparison
plt.figure(figsize=(18, 6))

# Original image
plt.subplot(1, 3, 1)
plt.imshow(lena)
plt.title('Original Lena Image')
plt.axis('off')

# Blurred image
plt.subplot(1, 3, 2)
plt.imshow(blurred_lena)
plt.title('Blurred Lena Image')
plt.axis('off')

# Sharpened image
plt.subplot(1, 3, 3)
plt.imshow(sharpened_lena)
plt.title('Sharpened Lena Image')
plt.axis('off')

plt.show()

# Print MSE and SSIM values
print(f'Mean Squared Error (MSE): {mse_value}')
print(f'Structural Similarity Index (SSIM): {ssim_value}')

### Part 2.2: Hybrid Images

In [ ]:
%matplotlib qt

# Define the function to create hybrid images
def hybrid_image(im1, im2, sigma1, sigma2, bw = False):
    # Step 1: Apply a low-pass filter (Gaussian blur) to the first image
    low_frequencies = cv2.GaussianBlur(im2, (0, 0), sigma2)
    
    # Step 2: Apply a high-pass filter to the second image by subtracting a blurred version from the original
    blurred_im1 = cv2.GaussianBlur(im1, (0, 0), sigma1)
    high_frequencies = im1 - blurred_im1

    # Step 3: Combine the two images (add them together)
    hybrid = np.clip(low_frequencies + high_frequencies, 0, 1)  # Ensure values stay in range [0, 1]
    
    # Convert to 8-bit if necessary
    if hybrid.dtype == np.float64:
        hybrid = (hybrid * 255).astype(np.uint8)  # Convert from float64 to uint8
    
    # If the output should be in grayscale
    if bw:
        hybrid = cv2.cvtColor(hybrid, cv2.COLOR_RGB2GRAY)

    return hybrid

# Define the function to plot pyramids (Gaussian and Laplacian pyramids)
def pyramids(image, num_levels):
    # Create Gaussian pyramid
    gaussian_pyramid = [image]
    for i in range(num_levels - 1):
        gaussian_pyramid.append(cv2.pyrDown(gaussian_pyramid[-1]))
    
    # Create Laplacian pyramid
    laplacian_pyramid = []
    for i in range(num_levels - 1):
        expanded = cv2.pyrUp(gaussian_pyramid[i+1])
        laplacian = cv2.subtract(gaussian_pyramid[i], expanded)
        laplacian_pyramid.append(laplacian)
    
    # Plot the pyramids
    fig, axes = plt.subplots(2, num_levels, figsize=(15, 5))
    
    for i in range(num_levels):
        axes[0, i].imshow(gaussian_pyramid[i], cmap='gray')
        axes[0, i].set_title(f'Gaussian Level {i+1}')
        axes[0, i].axis('off')
        
        if i < num_levels - 1:
            axes[1, i].imshow(laplacian_pyramid[i], cmap='gray')
            axes[1, i].set_title(f'Laplacian Level {i+1}')
            axes[1, i].axis('off')
    
    plt.show()


im1 = plt.imread('../data/jobs.jpg') / 255.  # Low-frequency image
im2 = plt.imread('../data/woz.jpg') / 255.  # High-frequency image

# Align the images
im1_aligned, im2_aligned = align_images(im1, im2)

sigma1 = 1  # High-pass filter (for im1)
sigma2 = 5  # Low-pass filter (for im2)

# Create the hybrid image
hybrid = hybrid_image(im1_aligned, im2_aligned, sigma1, sigma2, bw=True)

# Display the hybrid image
plt.imshow(hybrid, cmap='gray')
plt.title('Hybrid Image')
plt.axis('off')
plt.show()

# Generate Gaussian and Laplacian pyramids for the hybrid image
pyramids(hybrid, num_levels = 2)  # Adjust N to set the number of pyramid levels

### Fourier analysis

In [ ]:
%matplotlib inline

# Function to compute and display the log magnitude of the Fourier Transform in grayscale
def display_fft(image, title):
    # Convert the image to float32 if it's float64
    if image.dtype == np.float64:
        image = image.astype(np.float32)
    
    # Ensure the image is grayscale (if it's RGB)
    if len(image.shape) == 3 and image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # Compute the Fourier transform and shift zero frequency to the center
    fft = np.fft.fftshift(np.fft.fft2(image))
    magnitude_spectrum = np.log(np.abs(fft) + 1e-10)  # Avoid log(0)
    
    return magnitude_spectrum

# Main block to plot the Fourier Transforms
def plot_fourier_analysis(im1, im2, low_freq_image, high_freq_image, hybrid_image):
    # Prepare the figure for plotting
    fig, axes = plt.subplots(1, 5, figsize=(20, 4))

    # Titles for each image
    titles = ['Low Frequency Image', 'Low Frequency Filtered', 
              'High Frequency Image', 'High Frequency Filtered', 'Hybrid Image']
    
    # Compute FFT for each image
    ffts = [
        display_fft(im1, titles[0]),  # Low Frequency Image FFT
        display_fft(low_freq_image, titles[1]),  # Low Frequency Filtered FFT
        display_fft(im2, titles[2]),  # High Frequency Image FFT
        display_fft(high_freq_image, titles[3]),  # High Frequency Filtered FFT
        display_fft(hybrid_image, titles[4])  # Hybrid Image FFT
    ]
    
    # Plot each FFT result
    for ax, fft, title in zip(axes, ffts, titles):
        ax.imshow(fft, cmap='gray')
        ax.set_title(title)
        ax.axis('off')

    plt.show()

low_freq_image = cv2.GaussianBlur(im2_aligned, (0, 0), sigma2)  # Low-pass filtered image
high_freq_image = im1_aligned - cv2.GaussianBlur(im1_aligned, (0, 0), sigma1)  # High-pass filtered image

# Plot the Fourier analysis
plot_fourier_analysis(im1_aligned, im2_aligned, low_freq_image, high_freq_image, hybrid)

### Part 2.3: Gaussian and Laplacian Stacks

In [ ]:
# Load the image
apple = cv2.imread('../data/apple.jpeg')
orange = cv2.imread('../data/orange.jpeg')

def normalize_image(image):
    return (image - np.min(image)) / (np.max(image) - np.min(image))

def gaussian_stack(image, levels=5, sigma=1.0):
    image = image.astype(np.float32)
    gaussian_stack = [image]
    
    for i in range(1, levels):
        # create_gaussian_filter(sigma, kernel_size)
        blurred = cv2.GaussianBlur(gaussian_stack[i - 1], (0, 0), sigma)
        gaussian_stack.append(blurred)
    
    return gaussian_stack

def laplacian_stack(gaussian_stack):
    laplacian_stack = []
    
    for i in range(len(gaussian_stack) - 1):
        laplacian = gaussian_stack[i] - gaussian_stack[i + 1]
        laplacian_stack.append(laplacian)
    
    # Last level is the lowest frequency Gaussian image
    laplacian_stack.append(gaussian_stack[-1])
    
    return laplacian_stack

def process_image_by_channels(image, levels=5, sigma=1.0):
    channels = cv2.split(image)
    
    # Process each channel individually
    gaussian_stacks = [gaussian_stack(channel, levels, sigma) for channel in channels]
    laplacian_stacks = [laplacian_stack(gaussian_stack) for gaussian_stack in gaussian_stacks]
    
    # Stack the channels back together at each level
    gaussian_combined_stack = [cv2.merge([gaussian_stacks[0][i], gaussian_stacks[1][i], gaussian_stacks[2][i]]) 
                               for i in range(levels)]
    laplacian_combined_stack = [cv2.merge([laplacian_stacks[0][i], laplacian_stacks[1][i], laplacian_stacks[2][i]]) 
                                for i in range(levels)]
    
    return gaussian_combined_stack, laplacian_combined_stack

def visualize_stack(stack, title = "Stack"):
    plt.figure(figsize=(15, 10))
    
    for i, img in enumerate(stack):
        # Normalize the image for proper visualization
        img_normalized = normalize_image(img)
        plt.subplot(1, len(stack), i+1)
        plt.imshow(cv2.cvtColor(img_normalized, cv2.COLOR_BGR2RGB))  # Convert from BGR to RGB for display
        plt.title(f"{title} Level {i}")
        plt.axis('off')
    
    plt.show()

levels = 5
sigma = 2.0

# Generate stacks for the image
gaussian_stack_img, laplacian_stack_apple = process_image_by_channels(apple, levels=levels, sigma=sigma)
gaussian_stack_img, laplacian_stack_orange = process_image_by_channels(orange, levels=levels, sigma=sigma)

# Visualize the Laplacian stack
visualize_stack(laplacian_stack_apple, title="Laplacian Stack")
visualize_stack(laplacian_stack_orange, title="Laplacian Stack")

In [ ]:
def reconstruct_image_from_laplacian_stack(laplacian_stack):
    reconstructed_image = laplacian_stack[-1]
    
    for i in range(len(laplacian_stack) - 2, -1, -1):
        reconstructed_image = cv2.add(reconstructed_image, laplacian_stack[i])
        reconstructed_image = np.clip(reconstructed_image, 0, 255)  # Ensure pixel values are valid
    
    return reconstructed_image

reconstructed_apple = reconstruct_image_from_laplacian_stack(laplacian_stack_apple)
reconstructed_orange = reconstruct_image_from_laplacian_stack(laplacian_stack_orange)

def visualize_image(image, title="Reconstructed Image"):
    # Normalize the image for display purposes
    image_normalized = normalize_image(image)
    plt.imshow(cv2.cvtColor(image_normalized, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

# Visualize the reconstructed images
visualize_image(reconstructed_apple, title="Reconstructed Apple Image")
visualize_image(reconstructed_orange, title="Reconstructed Orange Image")


### Part 2.4: Multiresolution Blending

In [ ]:
# Load images
image1 = cv2.imread('../data/apple.jpeg')
image2 = cv2.imread('../data/orange.jpeg')

# Ensure images are the same size
image1 = cv2.resize(image1, (image2.shape[1], image2.shape[0]))

# Create a vertical mask
mask = np.zeros_like(image1, dtype=np.float32)
mask[:, :mask.shape[1]//2] = 1  # Left half is 1, right half is 0

# Display images and mask
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
plt.title('Image 1')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
plt.title('Image 2')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(mask, cmap='gray')
plt.title('Mask')
plt.axis('off')

plt.show()

In [ ]:
def gaussian_stack(image, levels, sigma):
    g_stack = [image.astype(np.float32)]
    for i in range(1, levels):
        blurred = cv2.GaussianBlur(g_stack[-1], (0, 0), sigma)
        g_stack.append(blurred)
    return g_stack

def laplacian_stack(g_stack):
    l_stack = []
    for i in range(len(g_stack) - 1):
        laplacian = g_stack[i] - g_stack[i + 1]
        l_stack.append(laplacian)
    l_stack.append(g_stack[-1])
    return l_stack

levels = 4
sigma = 1.0

# Create Gaussian and Laplacian stacks for images and mask
g_stack_image1 = gaussian_stack(image1, levels, sigma)
g_stack_image2 = gaussian_stack(image2, levels, sigma)
g_stack_mask = gaussian_stack(mask, levels, sigma = 64)

l_stack_image1 = laplacian_stack(g_stack_image1)
l_stack_image2 = laplacian_stack(g_stack_image2)

def blend_laplacian_stacks(l_stack1, l_stack2, g_stack_mask):
    blended_stack = []
    for i in range(len(l_stack1)):
        blended = l_stack1[i] * g_stack_mask[i] + l_stack2[i] * (1 - g_stack_mask[i])
        blended_stack.append(blended)
        
    return blended_stack

blended_l_stack = blend_laplacian_stacks(l_stack_image1, l_stack_image2, g_stack_mask)

def reconstruct_image(laplacian_stack):
    reconstructed_image = laplacian_stack[-1]
    for i in range(len(laplacian_stack) - 2, -1, -1):
        reconstructed_image = cv2.add(reconstructed_image, laplacian_stack[i])
    return reconstructed_image

blended_image = reconstruct_image(blended_l_stack)

# Display the blended image
plt.imshow(cv2.cvtColor(blended_image.astype(np.uint8), cv2.COLOR_BGR2RGB))
plt.title('Blended Image')
plt.axis('off')
plt.show()

In [20]:
def load_and_resize_images(img_path1, img_path2):
    image1 = cv2.imread(img_path1)
    image2 = cv2.imread(img_path2)
    image1 = cv2.resize(image1, (image2.shape[1], image2.shape[0]))
    return image1, image2

def create_horizontal_mask(image, offset = 0):
    mask = np.zeros_like(image, dtype=np.float32)
    mask[:mask.shape[0]//2 + offset, :] = 1  # Upper half is 1, lower half is 0
    return mask

def gaussian_stack(image, levels, sigma):
    g_stack = [image.astype(np.float32)]
    for i in range(1, levels):
        blurred = cv2.GaussianBlur(g_stack[-1], (0, 0), sigma)
        g_stack.append(blurred)
    return g_stack

def laplacian_stack(g_stack):
    l_stack = []
    for i in range(len(g_stack) - 1):
        laplacian = g_stack[i] - g_stack[i + 1]
        l_stack.append(laplacian)
    l_stack.append(g_stack[-1])
    return l_stack

def blend_images(image1, image2, mask, levels = 4, sigma = 1.0, sigma_mask = 64):
    g_stack_image1 = gaussian_stack(image1, levels, sigma)
    g_stack_image2 = gaussian_stack(image2, levels, sigma)
    g_stack_mask = gaussian_stack(mask, levels, sigma = sigma_mask)

    l_stack_image1 = laplacian_stack(g_stack_image1)
    l_stack_image2 = laplacian_stack(g_stack_image2)

    blended_l_stack = []
    for i in range(len(l_stack_image1)):
        blended = l_stack_image1[i] * g_stack_mask[i] + l_stack_image2[i] * (1 - g_stack_mask[i])
        blended_l_stack.append(blended)

    return reconstruct_image(blended_l_stack)

def reconstruct_image(laplacian_stack):
    reconstructed_image = laplacian_stack[-1]
    for i in range(len(laplacian_stack) - 2, -1, -1):
        reconstructed_image = cv2.add(reconstructed_image, laplacian_stack[i])
    return reconstructed_image

In [ ]:
# Apple and orange blending
image1, image2 = load_and_resize_images('../data/apple.jpeg', '../data/orange.jpeg')
mask = create_horizontal_mask(image1)

blended_image = blend_images(image1, image2, mask)

# Display the blended image
plt.imshow(cv2.cvtColor(blended_image.astype(np.uint8), cv2.COLOR_BGR2RGB))
plt.title('Blended Image')
plt.axis('off')
plt.show()

In [ ]:
# Cactus Ice Cream Blending
image1, image2 = load_and_resize_images('../data/cactus.jpg', '../data/ice_cream.jpg')
mask = create_horizontal_mask(image1)

blended_image = blend_images(image1, image2, mask, levels = 6, sigma = 0.4, sigma_mask = 2)

# Display the blended image
plt.imshow(cv2.cvtColor(blended_image.astype(np.uint8), cv2.COLOR_BGR2RGB))
plt.title('Blended Image')
plt.axis('off')
plt.show()

In [ ]:
def visualize_laplacian_stack(stack, title="Laplacian Stack"):
    plt.figure(figsize=(15, 10))
    for i, img in enumerate(stack):
        plt.subplot(1, len(stack), i+1)
        plt.imshow(cv2.cvtColor(normalize_image(img), cv2.COLOR_BGR2RGB))
        plt.title(f"{title} Level {i}")
        plt.axis('off')
    plt.show()

# Visualize the Laplacian stacks
visualize_laplacian_stack(blended_l_stack, title="Laplacian Stack Image 1")